In [59]:
import pandas as pd 
import numpy as np
import yfinance as yf

In [60]:
tickers = sorted(['AAPL', 'VZ', 'F', 'COKE'])
start = "2010-01-01"
end = "2019-12-31"

raw_stocks = yf.download(tickers,start,end).stack().reset_index()

raw_stocks

[*********************100%***********************]  4 of 4 completed
/var/folders/5k/s9c0d19s0k7byf19zlnzgmjr0000gn/T/ipykernel_43773/209562437.py:5: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  raw_stocks = yf.download(tickers,start,end).stack().reset_index()


Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2010-01-04 00:00:00+00:00,AAPL,6.454504,7.643214,7.660714,7.585000,7.622500,493729600
1,2010-01-04 00:00:00+00:00,COKE,46.175461,53.590000,54.230000,52.230000,53.279999,30600
2,2010-01-04 00:00:00+00:00,F,5.756343,10.280000,10.280000,10.050000,10.170000,60855800
3,2010-01-04 00:00:00+00:00,VZ,14.697927,31.212946,31.372387,31.034746,31.325493,16176648
4,2010-01-05 00:00:00+00:00,AAPL,6.465665,7.656429,7.699643,7.616071,7.664286,601904800
...,...,...,...,...,...,...,...,...
10055,2019-12-27 00:00:00+00:00,VZ,46.698357,61.529999,61.619999,61.250000,61.389999,8166800
10056,2019-12-30 00:00:00+00:00,AAPL,70.733627,72.879997,73.172501,71.305000,72.364998,144114400
10057,2019-12-30 00:00:00+00:00,COKE,277.493073,289.000000,293.179993,285.019989,293.179993,35600
10058,2019-12-30 00:00:00+00:00,F,7.513155,9.250000,9.350000,9.230000,9.340000,36074900


In [61]:
# Clean Data
df = raw_stocks.copy()

# Rename olumns
df = df.rename(columns={x:x.replace(" ","_").lower() for x in df.columns})

# Keep columns
keep_columns = ['date', 'ticker', 'close', 'adj_close']
df = df[keep_columns]

# Create ret column
df['ret'] = df.groupby('ticker')['adj_close'].pct_change()

# Sort dataframe
df = df.sort_values(by=['ticker','date'])

# Reindex
df = df.reset_index(drop=True)

df

Price,date,ticker,close,adj_close,ret
0,2010-01-04 00:00:00+00:00,AAPL,7.643214,6.454504,NaN
1,2010-01-05 00:00:00+00:00,AAPL,7.656429,6.465665,0.001729
2,2010-01-06 00:00:00+00:00,AAPL,7.534643,6.362821,-0.015906
3,2010-01-07 00:00:00+00:00,AAPL,7.520714,6.351058,-0.001849
4,2010-01-08 00:00:00+00:00,AAPL,7.570714,6.393281,0.006648
...,...,...,...,...,...
10055,2019-12-23 00:00:00+00:00,VZ,61.400002,46.599693,-0.010794
10056,2019-12-24 00:00:00+00:00,VZ,61.279999,46.508614,-0.001955
10057,2019-12-26 00:00:00+00:00,VZ,61.290001,46.516212,0.000163
10058,2019-12-27 00:00:00+00:00,VZ,61.529999,46.698357,0.003916


In [62]:
df.to_csv("data/data.csv", index=False)

In [63]:
expected_returns = df.groupby("ticker")['ret'].mean().to_numpy()

np.save('data/expected_returns.npy', expected_returns)

expected_returns

array([0.00108455, 0.00089106, 0.00025845, 0.00051336])

In [64]:
cov_matrix = df.pivot(index='date', values='ret', columns='ticker').fillna(0).cov().to_numpy()

np.save('data/covariance_matrix.npy',cov_matrix)

cov_matrix

array([[2.63071529e-04, 5.65198339e-05, 1.01122854e-04, 3.69110421e-05],
       [5.65198339e-05, 3.50739170e-04, 7.15853882e-05, 4.53049969e-05],
       [1.01122854e-04, 7.15853882e-05, 3.04050616e-04, 5.71982635e-05],
       [3.69110421e-05, 4.53049969e-05, 5.71982635e-05, 1.11028910e-04]])

In [65]:
df

Price,date,ticker,close,adj_close,ret
0,2010-01-04 00:00:00+00:00,AAPL,7.643214,6.454504,NaN
1,2010-01-05 00:00:00+00:00,AAPL,7.656429,6.465665,0.001729
2,2010-01-06 00:00:00+00:00,AAPL,7.534643,6.362821,-0.015906
3,2010-01-07 00:00:00+00:00,AAPL,7.520714,6.351058,-0.001849
4,2010-01-08 00:00:00+00:00,AAPL,7.570714,6.393281,0.006648
...,...,...,...,...,...
10055,2019-12-23 00:00:00+00:00,VZ,61.400002,46.599693,-0.010794
10056,2019-12-24 00:00:00+00:00,VZ,61.279999,46.508614,-0.001955
10057,2019-12-26 00:00:00+00:00,VZ,61.290001,46.516212,0.000163
10058,2019-12-27 00:00:00+00:00,VZ,61.529999,46.698357,0.003916


In [66]:
prices = df.groupby('ticker').agg({'close': 'last'}).to_numpy().T[0]

np.save('data/prices.npy', prices)

prices

array([ 72.87999725, 289.        ,   9.25      ,  61.20999908])